[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1NF6pDYVgjrb02dsIEcZAW4JVsiUKPXFB#scrollTo=xIWQ1hWAu25g)


# CRAWLING BERITA

## Rahma Nurhaliza (210411100176)

Teknik crawling adalah proses pengumpulan informasi dari berbagai sumber di internet secara otomatis. Web crawling biasanya dilakukan
oleh program yang disebut web crawler atau spider. Tujuan dari web crawling adalah untuk mengindeks konten di web sehingga dapat dianalisis.



## Install Library

In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install tqdm

## Import Library

- request digunakan untuk meminta respon pada situs yang akan dilakukan crawling data
- BeatifulSoup digunakan untuk melakukan ekstraksi halaman konten agar dapat mengambil isi-isi konten pada berita di website Kompas.com
- pandas digunakan untuk manipulasi dan analisis data. Memungkinkan untuk bekerja secara tabuler
- tqdm digunakan untuk menampilkan progress bar saat menjalankan loop atau iterasi yang memerlukan waktu lama, sangat berguna ketika ingin memantau seberapa jauh proses iterasi telah berjalan,

In [ ]:
from os import replace
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

## CRAWLING BERITA KOMPAS


Data berita berasal dari situs kompas, pada kategori IKN



In [14]:
def kompasnews(page, limit=15): #page: jumlh halaman yg ingin di crawl, limit: batas jumlah berita yang ingin diambil
    data = {'judul': [], 'berita': [], 'tanggal': []} #menyimpan informasi yang akan diambil
    article_count = 0 #menghitung berapa banyak artikel yang telah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break

        url = f"https://IKN.kompas.com/?source=navbar&page={i}" #(i) : nomor halaman saat ini, yaitu halaman yang akan dikunjungi crawler.
        r = requests.get(url)
        soup = BeautifulSoup(r.content, 'html.parser')

        # Menemukan elemen artikel
        articles = soup.find_all('div', {'class': 'articleItem'})
        if not articles:
            print(f" menemukan 0 artikel pada halaman {i}")
            continue

        for article in articles:
            if article_count >= limit:
                break

            # Ambil URL artikel
            a_element = article.find('a', {'class': 'article-link'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            # Ambil judul
            judul = article.find('h2', {'class': 'articleTitle'})
            judul = judul.text.strip() if judul else 'No title'

            # Ambil tanggal
            tanggal = article.find('div', {'class': 'articlePost-date'})
            tanggal = tanggal.text.strip() if tanggal else 'No date'

            # Ambil berita dari artikel detail
            berita_div = detail_soup.find('div', {'class': 'read__content'})
            berita = berita_div.text.strip() if berita_div else 'No content'

            # Tambahkan ke data
            data["judul"].append(judul)
            data["berita"].append(berita)
            data["tanggal"].append(tanggal)

            article_count += 1

    df = pd.DataFrame(data)
    df.to_csv('berita-kompas.csv', index=False)
    return df

In [15]:
kompasnews(1,limit=15)

100%|██████████| 1/1 [00:18<00:00, 18.44s/it]


,judul,berita,tanggal
0,"Angka Stunting di IKN 30 Persen, di Atas Rata-...","NUSANTARA, KOMPAS.com - Angka stunting di wila...",03/09/2024
1,Pj Gubernur Kaltim Minta Anggota DPRD 2024-202...,"NUSANTARA, KOMPAS.com - Penjabat (Pj) Gubernur...",03/09/2024
2,"Ruang buat Prabowo, Anggaran IKN dari Kementer...","NUSANTARA, KOMPAS.com - Pagu anggaran Kementer...",03/09/2024
3,"""Istana Kepresidenan IKN Harus Berwibawa, Ada ...","NUSANTARA, KOMPAS.com - Kompleks Istana Kepres...",02/09/2024
4,"Bom Waktu Air Bersih di Sekitar IKN, Pemerinta...","NUSANTARA, KOMPAS.com - Masalah air bersih yan...",02/09/2024
5,"Kebutuhan Air Rusun BIN, Polri dan ASN IKN Aka...","NUSANTARA, KOMPAS.com - Kebutuhan air bersih d...",02/09/2024
6,"Jalan Mulus Tak Hanya di IKN, Juga Akan Dibang...","NUSANTARA, KOMPAS.com - Jalan dalam kondisi ma...",02/09/2024
7,"Ibu Kota Pindah ke IKN, RocknMob Digelar 2 Kal...","NUSANTARA, KOMPAS.com - Aktivitas rocknmob ata...",02/09/2024
8,Nusantara Jadi Ibu Kota Negara Tunggu Keputusa...,"NUSANTARA, KOMPAS.com - Kepala Sekretariat Pre...",01/09/2024
9,"Warga Sekitar IKN Sulit Akses Air Bersih, Terk...","NUSANTARA, KOMPAS.com - Warga yang tinggal sej...",01/09/2024


## CRAWLING BERITA CNN

Data berita berasal dari situs CNN, pada kategori ekonomi


In [16]:
def cnnnews(page, limit=10):  # Tambahkan parameter limit dengan default 10
    data = {'judul': [], 'berita': [], 'tanggal': []}
    article_count = 0  # Menyimpan jumlah artikel yang sudah diambil

    for i in tqdm(range(1, page + 1)):
        if article_count >= limit:
            break  # Hentikan loop jika jumlah artikel sudah mencapai limit

        url = f"https://www.cnnindonesia.com/ekonomi/indeks/5/{i}"
        r = requests.get(url)
        request = r.content
        soup = BeautifulSoup(request, 'html.parser')
        soup = soup.find('div', {'class': 'flex flex-col gap-5'})
        news = soup.findAll('article', {'class': 'flex-grow'})

        for new in tqdm(news):
            if article_count >= limit:
                break  # Menghentikan loop jika jumlah artikel sudah mencapai limit

            a_element = new.find('a', {'aria-label': 'link description'})['href']
            detail_request = requests.get(a_element)
            detail_soup = BeautifulSoup(detail_request.content, 'html.parser')

            judul = detail_soup.find('h1', {'class': 'leading-9'})
            berita = detail_soup.find('div', {'class': 'detail-text'})
            tanggal = detail_soup.find('div', {'class': 'text-cnn_grey'})

            if judul and berita and tanggal:
                judul = judul.text
                berita = berita.text
                tanggal = tanggal.text
                noise = detail_soup.find('strong').text

                berita = berita.replace("ADVERTISEMENT", "").replace("SCROLL TO CONTINUE WITH CONTENT", "").replace("Lihat Juga :", "").replace("Bagikan", "").replace(noise, "").replace("-- ", "").replace("--", "")
                berita = ' '.join(berita.split())
                data["judul"].append(judul)
                data["berita"].append(berita)
                data["tanggal"].append(tanggal)

                article_count += 1  # penghitung artikel

    df = pd.DataFrame(data)
    df.to_csv("berita-cnn.csv", index=False)

    return df


In [17]:
cnnnews(2,limit=15)

100%|██████████| 2/2 [00:13<00:00,  6.80s/it]


,judul,berita,tanggal
0,Luhut Ingin Benahi Wisata Bali: Kelab Nude Ku...,Pemerintah pusat akan membenahi wisata di Bali...,"Selasa, 03 Sep 2024 13:50 WIB"
1,"Ma'ruf Amin: Potensi Zakat Rp270 T, Wakaf Rp1...",Wakil Presiden Ma'ruf Amin mengungkap banyak p...,"Selasa, 03 Sep 2024 13:34 WIB"
2,"Sepakati PKB, BRI Perkuat Hubungan Industrial...",PT Bank Rakyat Indonesia (Persero) Tbk atau BR...,"Selasa, 03 Sep 2024 12:52 WIB"
3,"Taji Ekonomi Vatikan, Negara Terkecil yang Di...",Kekuatan ekonomi Vatikan di bawah pimpinan Pau...,"Selasa, 03 Sep 2024 12:45 WIB"
4,"NeutraDC Kenalkan Smart Hands 2.0, Digital As...",Perusahaan NeutraDC yang menyediakan ekosistem...,"Selasa, 03 Sep 2024 12:42 WIB"
5,"Rosan: Investasi Indonesia di Afrika Menguat,...",Menteri Investasi/Kepala Badan Koordinasi Pena...,"Selasa, 03 Sep 2024 12:15 WIB"
6,"Tarif Tol BSD Bersiap Naik, Berikut Rinciannya",PT Bintaro Serpong Damai akan menaikkan tarif ...,"Selasa, 03 Sep 2024 12:10 WIB"
7,Blak-blakan Bappenas soal Sumber Uang Untuk M...,Kementerian Perencanaan Pembangunan Nasional/B...,"Selasa, 03 Sep 2024 11:53 WIB"
8,"Deflasi Melanda Ekonomi RI, Pertanda Bahaya A...",Alarm darurat ekonomi Indonesia kembali menyal...,"Selasa, 03 Sep 2024 11:19 WIB"
9,"Anggaran Kementerian BUMN Turun, Erick Thohir...",Menteri Badan Usaha Milik Negara (BUMN) Erick ...,"Selasa, 03 Sep 2024 10:53 WIB"
